In [1]:
import torch
import torch.nn as nn
from torch import optim as optim
from torchvision import models,datasets,transforms
import numpy as np
import pandas as pd
import cv2
import time
import torch.nn.functional as F
import torchsummary
import os
import glob
import pathlib
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
pwd

'C:\\Users\\Neil\\Documents\\ObsceneVideoDetector-master'

In [3]:
device= torch.device("cuda")

In [4]:
device

device(type='cuda')

In [5]:
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        
        self.resnet = models.resnet18(pretrained = True)
        k = 1
        for param in self.resnet.parameters():
            k+=1
            if k<56:
                param.requires_grad = False
        
        self.dense1 = nn.Linear(in_features =1000,out_features = 512)
        self.relu = nn.ReLU(inplace = True)
        self.dense2 = nn.Linear(in_features = 512,out_features = 256)
        self.dense3 = nn.Linear(in_features = 256,out_features = 64)
        self.out = nn.Linear(in_features = 64,out_features = 2)
        
    def forward(self,x):
        x = self.resnet(x)
        x = self.relu(x)
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        x = self.relu(x)
        x = self.dense3(x)
        x = self.relu(x)
        x = self.out(x)
        return F.softmax(x)

In [6]:
net = Network().to(device)

In [7]:
weights = torch.load("pytorch_image_modelling_70.pt",map_location = "cpu")

In [8]:
weights.keys()

dict_keys(['model', 'model_dict', 'optim_dict', 'epoch'])

In [9]:
net.load_state_dict(weights["model_dict"])

<All keys matched successfully>

In [10]:
optimizer = optim.Adam(net.parameters(),lr = 0.0005)

In [11]:
optimizer.load_state_dict(weights["optim_dict"])

In [12]:
criterion = nn.CrossEntropyLoss()

In [13]:
feature_extractor = nn.Sequential(*list(net.resnet.children())[:-2]).to(device)

In [14]:
torchsummary.summary(feature_extractor,input_size= (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

(tensor(11176512), tensor(4720640))

In [15]:
os.listdir("D:\\RESEARCH\\Dataset\\video_data")
path = "D:\\RESEARCH\\Dataset\\video_data"

In [16]:
train_add = os.path.join(path,"train")
val_add = os.path.join(path,"val")
test_add = os.path.join(path,"test")

In [22]:
train_add

'D:\\RESEARCH\\Dataset\\video_data\\train'

In [23]:
label_dict = {
    "porn":1,
    "non_porn":0
}

In [24]:
porn = "porn"
non_porn= "non_porn"

In [21]:
def AddressResolve(path,label_dict):
    porn_p = os.path.join(path,porn)
    nonporn_p = os.path.join(path,non_porn)
    p_folders = os.listdir(porn_p)
    np_folders = os.listdir(nonporn_p)
    folder_address_dict = {}
    for _item in p_folders:
        
        folder_address_dict[_item] = {}
        folder_address_dict[_item]["label"] = label_dict["porn"]
        folder_address_dict[_item]["address"] = os.path.join(porn_p,_item)
        frames= os.listdir(folder_address_dict[_item]["address"])
        frames = sorted(frames,key = lambda x: int(x.split("_")[1]))
        folder_address_dict[_item]["frames"] = [os.path.join(folder_address_dict[_item]["address"],x) for x in  frames]
    
    for _item in np_folders:
        
        folder_address_dict[_item] = {}
        folder_address_dict[_item]["label"] = label_dict["non_porn"]
        folder_address_dict[_item]["address"] = os.path.join(nonporn_p,_item)
        frames= os.listdir(folder_address_dict[_item]["address"])
        frames = sorted(frames,key = lambda x: int(x.split("_")[1]))
        folder_address_dict[_item]["frames"] = [os.path.join(folder_address_dict[_item]["address"],x) for x in  frames]

    return folder_address_dict    
         

In [38]:
train_dict = AddressResolve(train_add,label_dict)
test_dict = AddressResolve(test_add,label_dict)
val_dict = AddressResolve(val_add,label_dict)

In [44]:
pwd

'C:\\Users\\Neil\\Documents\\ObsceneVideoDetector-master'

In [47]:
torch.save(train_dict,"train_dict.pt")
torch.save(test_dict,"test_dict.pt")
torch.save(val_dict,"val_dict.pt")

In [16]:
train_dict = torch.load("train_dict.pt")
test_dict = torch.load("test_dict.pt")
val_dict = torch.load("val_dict.pt")

In [20]:
train_dict["frames_vPorn219"]

{'label': 1,
 'address': 'D:\\RESEARCH\\Dataset\\video_data\\train\\porn\\frames_vPorn219',
 'frames': ['D:\\RESEARCH\\Dataset\\video_data\\train\\porn\\frames_vPorn219\\vPorn219_1_1_1.jpg',
  'D:\\RESEARCH\\Dataset\\video_data\\train\\porn\\frames_vPorn219\\vPorn219_2_1_2.jpg',
  'D:\\RESEARCH\\Dataset\\video_data\\train\\porn\\frames_vPorn219\\vPorn219_3_1_3.jpg',
  'D:\\RESEARCH\\Dataset\\video_data\\train\\porn\\frames_vPorn219\\vPorn219_4_1_4.jpg',
  'D:\\RESEARCH\\Dataset\\video_data\\train\\porn\\frames_vPorn219\\vPorn219_5_2_1.jpg',
  'D:\\RESEARCH\\Dataset\\video_data\\train\\porn\\frames_vPorn219\\vPorn219_6_2_2.jpg',
  'D:\\RESEARCH\\Dataset\\video_data\\train\\porn\\frames_vPorn219\\vPorn219_7_2_3.jpg',
  'D:\\RESEARCH\\Dataset\\video_data\\train\\porn\\frames_vPorn219\\vPorn219_8_2_4.jpg',
  'D:\\RESEARCH\\Dataset\\video_data\\train\\porn\\frames_vPorn219\\vPorn219_9_3_1.jpg',
  'D:\\RESEARCH\\Dataset\\video_data\\train\\porn\\frames_vPorn219\\vPorn219_10_3_2.jpg',
  'D:\\

In [18]:
data_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [49]:
def extract_feature(add_list,name,target_path,model= feature_extractor):
    images_list = []
    for i in range(len(add_list)):
        image = Image.open(add_list[i])
        image = data_transform(image)
        image = image.view(1,3,224,224).to(device)
        with torch.no_grad():
            image_feature = feature_extractor(image)
            image_feature = image_feature.to(torch.device("cpu"))
            image_feature = image_feature.numpy()
            images_list.append(image_feature)
    images_list = np.array(images_list)  
    images_list = images_list.reshape(-1,512,7,7)
    images_list = torch.FloatTensor(images_list)
    save_dict = {name:images_list}
    torch.save(save_dict,os.path.join(target_path,name)+".pt")
    print(images_list.shape)
    torch.cuda.empty_cache()
    

In [ ]:
for key in train_dict.keys():
    if "vPorn" in key:
        extract_feature(train_dict[key]["frames"],key, "D:\\RESEARCH\\Dataset\\extracted_frame_features\\train\\porn")
    elif "vNonPorn" in key:
        extract_feature(train_dict[key]["frames"],key,"D:\\RESEARCH\\Dataset\\extracted_frame_features\\train\\non_porn")
